# AI Agents and Workflows
## Building Intelligent Systems with Spring AI

**Spring AI Workshop - Session #3**
*November 20, 2025*

**Speaker:** Ketevan Khachapuridze

## Agenda

1. **Core Concepts:** Agents, Workflows, and Building Blocks
2. **Decision Guide:** When to Use What
3. **Workflow Patterns:**
   - Chain Workflow
   - Routing Workflow
   - Parallelisation Workflow
   - Orchestrator-Workers
   - Evaluator-Optimizer
4. **Autonomous Agents**
5. **Building Reliable Systems:** Testing and Guardrails
6. **VoyagerMate Case Study:** Applied Patterns


## Core Concepts: Agents vs Workflows

[Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents)

**Agentic Systems** is an umbrella term for systems where LLMs actively reason, plan, and use tools to solve problems. These systems generally fall into two categories: **Workflows** and **Agents**.

### Workflows
Workflows are systems where the steps are predefined. The path is predictable and orchestrated by code.
- **You** decide the steps.
- **Code** controls the flow.
- The LLM **executes** each step.
- They are **reliable** and **testable**.

### Agents
Agents are systems where the LLM dynamically controls the process.
- The **LLM** decides the steps.
- The **LLM** controls the flow.
- The LLM **plans** and **adapts**.
- They are **flexible** but **less predictable**.

### The Impact of Reasoning Models
Modern "Reasoning Models" (like OpenAI o1 or Claude 3.7) have internalized many complex behaviors, using "Chain of Thought" to think before they answer.
- **Micro-Reasoning:** You often do not need complex prompts just to get the model to think step-by-step. The model does this automatically.
- **Self-Correction:** These models can often catch their own errors.

However, workflows remain essential for **Macro-Orchestration**: breaking down tasks that are too large for a single context window, require parallel execution, or involve distinct external actions.

### The Building Blocks

Both workflows and agents are built from the same foundation: an LLM enhanced with three key capabilities.

1. **Retrieve:** Generate search queries to find information.
2. **Use Tools:** Call external functions to act on the world.
3. **Remember:** Read and write context to memory.


```mermaid
flowchart LR
    User[User] --> LLM["LLM / Model"]
    subgraph Capabilities
        LLM <--> Tools[Tools]
        LLM <--> Memory[Memory]
        LLM <--> RAG["Retrieval / RAG"]
    end
```


## Decision Guide: When to Use Which?

> **Golden Rule:** Start simple. Only add autonomy when it is necessary.

### Use Workflows when...
You know the steps in advance. It is like following a recipe.
- The sequence of steps is fixed.
- Inputs and outputs are predictable.
- You need the same result every time.

**Example:** Converting user input to structured data. The steps (extract, validate, save) are always the same.

### Use Agents when...
The path depends on what you discover. It is like a detective investigating a case.
- You do not know the steps beforehand.
- Each action depends on previous findings.
- The solution requires exploration.

**Example:** A code debugging assistant. The agent must read the error, search code, and test hypotheses based on what it finds.

### Strategic Insights
- **Start with deterministic workflows** before escalating to autonomous agents.
- **Reliability levers** include tool design discipline, memory externalisation (vector or relational stores), and proactive context compaction to avoid "context rot."



## Pattern 1: Chain Workflow

**Prompt chaining** breaks a complex task into smaller, ordered steps. Each step uses the result of the previous one.

### When to Use
While reasoning models handle internal logic well, this pattern is ideal when:
- Steps involve **external actions** (e.g., API calls) that must happen in order.
- You need to **checkpoint state** or validate output between steps.
- You need to force a specific, audible process structure.


```mermaid
flowchart LR
    Input[Input] --> Step1["Step 1: Extract"]
    Step1 --> Step2["Step 2: Validate"]
    Step2 --> Step3["Step 3: Summarize"]
    Step3 --> Output[Output]
```


### Spring AI Implementation


In [ ]:
public class ChainWorkflow {
    private final ChatClient chatClient;
    private final String[] systemPrompts;

    // Processes input through a series of prompts,
    // where the output of each step becomes the input for the next.
    public String chain(String userInput) {
        String response = userInput;
        for (String prompt : systemPrompts) {
            // Combine the system prompt with previous response
            String input = String.format("{%s}\n{%s}", prompt, response);
            // Process through the LLM and capture output
            response = chatClient.prompt(input).call().content();
        }
        return response;
    }
}


## Pattern 2: Routing Workflow

**Routing** uses an LLM to classify an input and direct it to the correct specialized process. Instead of one model handling everything, a router selects the best tool or workflow.

### When to Use
- **Cost Optimization:** Route difficult tasks to expensive reasoning models and simple tasks to cheaper, faster models.
- **Specialization:** Different tools or models perform better on specific subtasks.
- **Separation of Concerns:** Separate decision-making from execution.


```mermaid
flowchart LR
    Input[Input] --> Router["Router LLM"]
    Router -->|Intent A| RouteA["Specialist A"]
    Router -->|Intent B| RouteB["Specialist B"]
    Router -->|Intent C| RouteC["Specialist C"]
```


### Spring AI Implementation


In [ ]:
@Autowired
private ChatClient chatClient;

// Create the workflow
RoutingWorkflow workflow = new RoutingWorkflow(chatClient);

// Define specialized prompts for each category
Map<String, String> routes = Map.of(
    "billing", "You are a billing specialist. Help resolve billing issues...",
    "technical", "You are a technical support engineer. Help solve technical problems...",
    "general", "You are a customer service representative. Help with general inquiries..."
);

// Route the input
String input = "My account was charged twice last week";
String response = workflow.route(input, routes);


## Pattern 3: Parallelization Workflow

**Parallelization** runs multiple LLM calls at the same time. The results are then combined or compared. This is useful for "Sectioning" (splitting a task) or "Voting" (getting multiple opinions).

### When to Use
- **Latency Reduction:** Reasoning models can be slow. Parallelization offsets this by running independent tasks concurrently.
- **Scale:** The task can be split into independent parts.
- **Confidence:** Multiple viewpoints increase accuracy (voting).


```mermaid
flowchart LR
    Input[Input] --> Split{Split}
    Split --> TaskA["Task A"]
    Split --> TaskB["Task B"]
    Split --> TaskC["Task C"]
    TaskA --> Join{Aggregator}
    TaskB --> Join
    TaskC --> Join
    Join --> Output[Output]
```


### Spring AI Implementation


In [ ]:
List<String> parallelResponse = new ParallelizationWorkflow(chatClient)
    .parallel(
        "Analyze how market changes will impact this stakeholder group.",
        List.of(
            "Customers: ...",
            "Employees: ...",
            "Investors: ...",
            "Suppliers: ..."
        ),
        4
    );


## Pattern 4: Orchestrator-Workers

In this pattern, a central **Orchestrator LLM** breaks a task into subtasks and assigns them to **Worker LLMs**. The orchestrator then synthesizes the results. Unlike parallelization, the orchestrator dynamically decides the plan.

### When to Use
- The task is complex (e.g., "Write a full software module") and exceeds a single model's capacity.
- The subtasks cannot be predicted in advance.
- You need to combine results from different perspectives or tools.


```mermaid
flowchart TD
    Task["Complex Task"] --> Orchestrator["Orchestrator LLM"]
    Orchestrator -->|Plan & Delegate| Worker1["Worker 1"]
    Orchestrator -->|Plan & Delegate| Worker2["Worker 2"]
    Worker1 -->|Result| Orchestrator
    Worker2 -->|Result| Orchestrator
    Orchestrator -->|Synthesize| Output["Final Result"]
```


### Spring AI Implementation


In [ ]:
public class OrchestratorWorkersWorkflow {

    public WorkerResponse process(String taskDescription) {
        // 1. Orchestrator analyzes task and determines subtasks
        OrchestratorResponse orchestratorResponse = // ...

        // 2. Workers process subtasks in parallel
        List<String> workerResponses = // ...

        // 3. Results are combined into a final response
        return new WorkerResponse(/* ... */);
    }
}


## Pattern 5: Evaluator-Optimizer

This workflow creates a feedback loop. One LLM (the **Generator**) creates a draft, and another LLM (the **Evaluator**) reviews it. The Generator then improves the draft based on the feedback.

### When to Use
- **External Verification:** Reasoning models self-correct well, but this pattern is vital for checking against **external ground truths** (e.g., "Code must compile", "Policy check passed").
- You have clear, objective criteria for quality.
- Iteration typically improves the result.


```mermaid
flowchart LR
    Input[Task] --> Generator["Generator LLM"]
    Generator --> Draft[Draft]
    Draft --> Evaluator["Evaluator LLM"]
    Evaluator -->|Feedback| Generator
    Evaluator -->|Approved| Output["Final Output"]
```


### Spring AI Implementation


In [ ]:
public class EvaluatorOptimizerWorkflow {
    public RefinedResponse loop(String task) {
        // 1. Generate initial solution
        Generation generation = generate(task, context);
        
        // 2. Evaluate the solution
        EvaluationResponse evaluation = evaluate(generation.response(), task);
        
        // 3. If PASS, return solution
        // 4. If NEEDS_IMPROVEMENT, incorporate feedback and generate new solution
        // 5. Repeat until satisfactory
        return new RefinedResponse(finalSolution, chainOfThought);
    }
}


## Autonomous Agents

Agents handle tasks that do not fit a fixed workflow. An agent starts with a goal, creates a plan, acts, and observes the results. It loops through this process until the goal is met.

### When to Use
- The task is open-ended.
- The number of steps is variable.
- The environment provides feedback (like code execution or API results).

## Multi-Agent Systems

Multi-agent systems use teams of specialized agents. Each agent focuses on one job (e.g., research, writing, review) and they coordinate to solve larger problems.


```mermaid
flowchart TD
    User[User] --> AgentA["Agent A (Leader)"]
    AgentA <--> AgentB["Agent B (Researcher)"]
    AgentA <--> AgentC["Agent C (Writer)"]
    AgentB <--> AgentC
```


## Building Reliable Systems

Building effective agentic systems requires more than just prompts. You need a solid engineering foundation.

### Spring AI Advantages
- **Model Portability:** Switch models easily using dependencies.
- **Structured Output:** Map LLM responses directly to Java objects (DTOs).
- **Consistent API:** Use a uniform interface for different providers.

### Guardrails and Testing
You must ensure your system is safe and predictable.

1.  **Guardrails:** Define what the system can and cannot do.
    -   Use **Structured Outputs** to enforce response formats.
    -   Sanitize inputs and outputs.
    -   Set rate limits and retry policies.
2.  **Testing:**
    -   **Unit Tests:** Test routing and logic components.
    -   **Prompt Tests:** Verify LLM behavior with test suites.
    -   **Golden Responses:** Compare outputs against known good examples.
3.  **Observability:** Use structured logging to track tool calls and iterations.

### Key Recommendations
1.  **Start Simple:** Begin with basic workflows. Add complexity only when needed.
2.  **Use Reasoning Models for Logic:** Let the model handle "micro-reasoning" internally. Use workflows for "macro-orchestration."
3.  **Design Good Tools:** Ensure your tools are well-documented and reliable.
4.  **Balance Trade-offs:** More iterations improve quality but increase latency and cost.


```mermaid
graph TD
    Start["Start Simple"] --> Analyze{Needs?}
    Analyze -->|Predictable| Workflow[Workflow]
    Analyze -->|Open-ended| Agent[Agent]
    Workflow --> Test["Test & Iterate"]
    Agent --> Guardrails["Add Guardrails"]
    Guardrails --> Test
```


## VoyagerMate Case Study: Applied Patterns

We have implemented these patterns in **VoyagerMate** to handle the complexity of travel planning.

### 1. Chain Workflow (The Foundation)
**Implementation:** `ItineraryWorkflowService`
We use a four-step deterministic chain (Identify -> Plan -> Budget -> Summary). This ensures every trip plan has these essential components regardless of the LLM's creative variations. It is the reliable backbone of the system.

### 2. Routing Workflow (The Concierge)
**Implementation:** `VoyagerRoutingWorkflowService`
We classify user prompts into intents like `CONCIERGE`, `BOOKING_CHANGE`, or `TRAVEL_RISK`. This allows us to use a fast model for simple chats and a deeper reasoning model (or a specialized prompt) for risk assessment or complex modifications.

### 3. Parallelization Workflow (The Research Sweep)
**Implementation:** `ParallelItineraryWorkflowService`
Trip planning involves checking lodging, dining, and logistics simultaneously. We use Java's `CompletableFuture` and Virtual Threads to run concurrent research tracks. This significantly reduces the total latency compared to checking each category sequentially.

### 4. Orchestrator-Workers (The Planner)
**Implementation:** `OrchestratorWorkersWorkflowService`
For vague requests like "Plan a sabbatical," we cannot use a fixed chain. An Orchestrator LLM breaks the brief down into specific subtasks (Visa Check, Accommodation, Experiences) and delegates them to specialized Worker LLMs (via `VoyagerTools`). It then synthesizes their findings into a cohesive narrative.

### 5. Evaluator-Optimizer (The Reviewer)
**Implementation:** `ItineraryRefinementWorkflowService`
When generating detailed itineraries, quality matters. We use a generator model to create a draft, and a separate "Critic" model to review it for feasibility and budget constraints. The generator refines the plan based on this feedback, ensuring the final output is realistic.

### 6. Multi-Agent System (The Team)
**Implementation:** `MultiAgentOrchestratorService`
We use the **Agents-as-Tools** pattern to coordinate a team of specialists. A Lead Orchestrator LLM is equipped with specialized tools (`ask_logistics_expert`, `ask_accommodation_expert`, `ask_activity_expert`). When the Orchestrator needs specific details, it calls these tools, which in turn spin up their own specialized LLM sessions. The Orchestrator then synthesizes these expert contributions into a final plan. This demonstrates how to compose multiple agents into a cohesive system using standard tool-calling mechanisms.
